In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.adapt import MLkNN
import sklearn.metrics as metrics
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import multilabel_confusion_matrix
import os

In [2]:
d2= pd.read_csv('train_hi_l1.csv')
d2
d2 = d2.rename(columns={'key' : 'unique_id', 'sentence' : 'text'})
d2.to_csv('updated_train_hi_l1.csv', index=False)
# d2

# Convert annotator columns to numeric without replacing NaNs
d2[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']] = d2[
    ['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']
].apply(pd.to_numeric, errors='coerce')

# Compute 'label' from Hindi annotators using majority voting (ignore NaNs)
d2['label'] = (d2[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']].mean(axis=1, skipna=True) >= 0.5).astype(int)
d2


,text,unique_id,hi_a1,hi_a2,hi_a3,hi_a4,hi_a5,label
0,भारत में तीन किसान कानून बिल वापस लेने पर भक...,question_1,NaN,NaN,NaN,0.0,NaN,0
1,राजस्थान अजीबोगरीब: गांव की खुशहाली के लिए ...,question_1,NaN,0.0,NaN,NaN,NaN,0
2,सलमान ने की राखी के विरोधी की वकालत तो 'आइटम...,question_1,NaN,NaN,NaN,NaN,0.0,0
3,!!हर शब्द अमॄतम!! पुरानी एक कहावत है... टूटी ...,question_1,NaN,NaN,NaN,0.0,NaN,0
4,"""PM मोदी जी "" की माँ के लिए अपशब्द कहने वाला द...",question_1,NaN,1.0,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...
6192,"😎 काम वाली बाई का फोन आया, पत्नि को। कामवाली ...",question_1,NaN,NaN,NaN,0.0,NaN,0
6193,😎😎ग्रुप में चर्चा चल रही थी कि 21 दिन के लॉक ड...,question_1,NaN,0.0,NaN,NaN,NaN,0
6194,🚩 🔱▁▂▄▅▆▇█🦁█▇▆▅▄▂▁🔱 🔱⚜️👉 School में हमें स...,question_1,NaN,NaN,NaN,0.0,NaN,0
6195,"🚩 #लंका का जिहादी डॉक्टर,हजारों हिंदू महिलाओं ...",question_1,NaN,NaN,NaN,NaN,0.0,0


In [3]:
# Create binary label ('hate' or 'not_hate')
def determine_binary_label(label):
    return 'hate' if label == 1 else 'not_hate'

d2['binary_label'] = d2['label'].apply(determine_binary_label)

# # Reorder columns
d2 = d2[['unique_id', 'text', 'binary_label', 'label']]

In [4]:
d2.to_csv('updated_train_hi_l1.csv', index=False)

In [5]:
import nltk
import string
import re

def normalize_text(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F700-\U0001F77F"
                               u"\U0001F780-\U0001F7FF"
                               u"\U0001F800-\U0001F8FF"
                               u"\U0001F900-\U0001F9FF"
                               u"\U0001FA00-\U0001FA6F"
                               u"\U0001FA70-\U0001FAFF"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = text.lower()
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'<.*?>+', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', ' ', text)
    text = re.sub(r'<handle replaced>', '', text)
    text = emoji_pattern.sub(r'', text)
    return text

## Apply the written function ##
d2.loc[:, 'text'] = d2['text'].apply(lambda x: normalize_text(x))
processed_list = []
for j in d2['text']:
    process = j.replace('...','')
    processed_list.append(process)

df_processed = pd.DataFrame(processed_list)
df_processed.columns = ['text']
df_processed.head(n=5)

,text
0,भारत में तीन किसान कानून बिल वापस लेने पर भक...
1,राजस्थान अजीबोगरीब गांव की खुशहाली के लिए ...
2,सलमान ने की राखी के विरोधी की वकालत तो आइटम...
3,हर शब्द अमॄतम पुरानी एक कहावत है टूटी ...
4,pm मोदी जी की माँ के लिए अपशब्द कहने वाला द...


In [6]:
X = list(df_processed['text'])
y = d2[['label']].values
y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], shape=(6197, 1))

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Activation, Dropout, Dense, Flatten,
    Bidirectional, GRU, concatenate, SpatialDropout1D,
    GlobalMaxPooling1D, GlobalAveragePooling1D, Conv1D,
    Embedding, Input, Concatenate
)
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import MeanSquaredError

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

######## Textual Features for Embedding ###################
max_len = 100
max_features = 4479

# Tokenization
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding
X = pad_sequences(X, padding='post', maxlen=max_len)

print(X)  # Check the processed sequences

[[  63    5  896 ...    0    0    0]
 [ 629 1047    3 ...    0    0    0]
 [1783   21    3 ...    0    0    0]
 ...
 [   5  357 3995 ...    0    0    0]
 [   6  201 1216 ...    0    0    0]
 [ 205  205  688 ...    0    0    0]]


In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y.ravel())
y

array([0, 0, 0, ..., 0, 0, 0], shape=(6197,))

In [9]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=2)
y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], shape=(6197, 2))

In [10]:
import numpy as np
import json

# Load GloVe embeddings from JSON
with open('glove_embeddings.json', encoding="utf8") as f:
    embeddings_list = json.load(f)

# Convert the list of vectors to a dictionary with word indices as keys
embeddings_dictionary = {str(i): vector for i, vector in enumerate(embeddings_list)}

# Define tokenizer 
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size
word_index = tokenizer.word_index
num_words = min(max_features, vocab_size)  # Limit vocab to max_features

# Get embedding dimension (from first vector in list)
embed_size = len(embeddings_list[0]) if embeddings_list else 0

# Initialize embedding matrix
embedding_matrix = np.zeros((num_words, embed_size))

# Fill embedding matrix with corresponding word vectors
for word, index in word_index.items():
    if index >= max_features:
        continue
    embedding_vector = embeddings_dictionary.get(word) or embeddings_dictionary.get(str(index))
    if embedding_vector is not None:
        embedding_matrix[index] = np.asarray(embedding_vector, dtype=np.float32)

print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: (4479, 50)


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, Embedding, SpatialDropout1D, Conv1D,
    Bidirectional, LSTM, GRU, Dense, Dropout,
    GlobalAveragePooling1D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configure GPU for optimal performance
def configure_gpu():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Enable memory growth for each GPU
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
            # Use mixed precision for better performance
            policy = tf.keras.mixed_precision.Policy('mixed_float16')
            tf.keras.mixed_precision.set_global_policy(policy)
            print('Mixed precision enabled')
        except RuntimeError as e:
            print(e)

configure_gpu()

# Model Definition - GRU with Attention
def create_gru_attention_model(max_len, max_features, embedding_matrix, embed_size=300):
    """
    Creates an enhanced GRU model with hierarchical attention mechanism
    """
    # Input layer
    input_layer = Input(shape=(max_len,))
    
    # Embedding layer with pretrained weights
    embedding_layer = Embedding(
        input_dim=max_features,
        output_dim=embed_size,
        weights=[embedding_matrix],
        input_length=max_len,
        trainable=True  # Make embeddings trainable for fine-tuning
    )(input_layer)
    
    # Spatial Dropout with higher rate
    spatial_dropout = SpatialDropout1D(0.3)(embedding_layer)
    
    # Multiple GRU layers with different window sizes
    gru_layer1 = Bidirectional(
        GRU(
            units=128,
            return_sequences=True,
            dropout=0.2,
            recurrent_dropout=0.2,
            kernel_regularizer=tf.keras.regularizers.l2(1e-5)
        )
    )(spatial_dropout)
    
    gru_layer2 = Bidirectional(
        GRU(
            units=64,
            return_sequences=True,
            dropout=0.2,
            recurrent_dropout=0.2
        )
    )(gru_layer1)
    
    # Multi-head self-attention (simplified version)
    attention_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=8,
        key_dim=16
    )(gru_layer2, gru_layer2)
    
    # Skip connection
    concat_layer = tf.keras.layers.Concatenate()([gru_layer2, attention_layer])
    
    # Feature extraction with pooling operations
    avg_pool = GlobalAveragePooling1D()(concat_layer)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(concat_layer)
    
    # Combine pooled features
    concat_pools = tf.keras.layers.Concatenate()([avg_pool, max_pool])
    
    # Deep MLP layers with batch normalization and more dropout
    x = Dense(256, activation='relu')(concat_pools)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # Output layer
    output_layer = Dense(2, activation='softmax', dtype='float32')(x)
    
    # Create model
    model = Model(inputs=input_layer, outputs=output_layer)
    
    return model

# Custom macroF1 Score Metric
class MacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes = 2, name='macro_f1_score', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fp = self.add_weight(name='fp', initializer='zeros')
        self.fn = self.add_weight(name='fn', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert probabilities to predicted class indices
        y_pred = tf.argmax(y_pred, axis=-1)
        
        # Convert one-hot encoded y_true to class indices if needed
        if len(y_true.shape) > 1 and y_true.shape[-1] > 1:
            y_true = tf.argmax(y_true, axis=-1)
        
        # Initialize confusion matrix
        conf_matrix = tf.math.confusion_matrix(
            y_true,
            y_pred,
            num_classes=self.num_classes,
            dtype=tf.float32
        )
        
        # Calculate TP, FP, FN for each class
        diag = tf.linalg.diag_part(conf_matrix)
        row_sum = tf.reduce_sum(conf_matrix, axis=1)
        col_sum = tf.reduce_sum(conf_matrix, axis=0)
        
        tp = diag
        fp = col_sum - diag
        fn = row_sum - diag
        
        # Update the state variables
        self.tp.assign_add(tf.reduce_sum(tp))
        self.fp.assign_add(tf.reduce_sum(fp))
        self.fn.assign_add(tf.reduce_sum(fn))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        # Calculate precision and recall
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        
        # Calculate F1 score
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        
        # Return macro F1 (average of per-class F1 scores)
        return f1

    def reset_states(self):
        self.tp.assign(0.)
        self.fp.assign(0.)
        self.fn.assign(0.)
        self.count.assign(0.)
            
# Model Training
def train_and_validate_model(model, X_train, y_train, X_val, y_val, batch_size=32, epochs=15, model_dir='models_hi_task1_m2'):
    """
    Trains the GRU-Attention model with early stopping and model checkpointing
    Returns the best model and training history
    """
    # Create directory for saving models if it doesn't exist
    os.makedirs(model_dir, exist_ok=True)
    
    # Callbacks
    early_stopping = EarlyStopping(
        monitor='macro_f1_score',
        patience=2,
        restore_best_weights=True,
        mode='max',
        verbose=1
    )
    
    model_checkpoint = ModelCheckpoint(
        os.path.join(model_dir, 'best_model_hi_task1_m2.h5'),  # Save entire model
        monitor='macro_f1_score',
        mode='max',
        save_best_only=True,
        verbose=1
    )
    
    # Compile model with Adam optimizer (as per paper)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', MacroF1Score(num_classes=2)]
    )
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[early_stopping, model_checkpoint],
        verbose=1
    )
    
    # Load the best model found during training
    best_model = load_model(os.path.join(model_dir, 'best_model_hi_task1_m2.h5'), 
                          custom_objects={'MacroF1Score': MacroF1Score})
    
    return history, best_model

# Plot Training History
def plot_training_history(history, plot_dir='plots_nlp_project_hi_task1_m2'):
    """
    Plots training history (accuracy and loss curves)
    Saves plots to specified directory
    """
    os.makedirs(plot_dir, exist_ok=True)
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, 'training_history_hi_task1_m2.png'))
    plt.close()

# Validation Evaluation
def evaluate_validation(model, X_val, y_val, plot_dir='plots_nlp_project_hi_task1_m2'):
    """
    Evaluates the model on validation data and saves metrics and plots
    """
    os.makedirs(plot_dir, exist_ok=True)
    
    # Predict probabilities
    y_pred_proba = model.predict(X_val, batch_size=32)
    
    # Convert to class labels
    y_pred = np.argmax(y_pred_proba, axis=1)
    y_true = np.argmax(y_val, axis=1)
    
    # Calculate metrics
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    weighted_f1 = f1_score(y_true, y_pred, average='weighted')
    macro_f1 = f1_score(y_true, y_pred, average='macro')

    
    # Classification report
    report = classification_report(y_true, y_pred, target_names=['not_hate', 'hate'])
    
    # Confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Hate', 'Hate'],
                yticklabels=['Not Hate', 'Hate'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix (Validation)')
    plt.savefig(os.path.join(plot_dir, 'confusion_matrix_val_hi_task1_m2.png'))
    plt.close()
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score_weighted': weighted_f1,
        'f1_score_macro': macro_f1,
        'classification_report': report,
        'confusion_matrix': conf_matrix
    }


# Main Execution for Training and Validation
if __name__ == "__main__":
    # Split into train (80%) and validation (20%)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"Training samples: {len(X_train)}")
    print(f"Validation samples: {len(X_val)}")
    
    # Create model - using GRU with Attention instead of CNN-BiLSTM
    embed_size = embedding_matrix.shape[1]
    model = create_gru_attention_model(max_len, max_features, embedding_matrix, embed_size)
    
    # Print model summary
    model.summary()
    
    # Train model
    history, trained_model = train_and_validate_model(
        model, X_train, y_train, X_val, y_val,
        batch_size=32,
        epochs=15  
    )
    
    # Plot training history
    plot_training_history(history)
    
    # Evaluate on validation set
    val_results = evaluate_validation(trained_model, X_val, y_val)
    
    print("\nValidation Results:")
    print(f"Precision: {val_results['precision']:.4f}")
    print(f"Recall: {val_results['recall']:.4f}")
    print(f"weighted F1 Score: {val_results['f1_score_weighted']:.4f}")
    print(f"macro F1 Score: {val_results['f1_score_macro']:.4f}")
    print("\nClassification Report:")
    print(val_results['classification_report'])

Training samples: 4957
Validation samples: 1240


C:\Users\krmri\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 100)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 100, 50)           │         223,950 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ spatial_dropout1d             │ (None, 100, 50)           │               0 │ embedding[0][0]            │
│ (SpatialDropout1D)            │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional (Bidirectional) │ (None, 100, 256)          │         138,240 │ spatial_dropout1d[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_1               │ (None, 100, 128)          │         123,648 │ bidirectional[0][0]        │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention          │ (None, 100, 128)          │          66,048 │ bidirectional_1[0][0],     │
│ (MultiHeadAttention)          │                           │                 │ bidirectional_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 100, 256)          │               0 │ bidirectional_1[0][0],     │
│                               │                           │                 │ multi_head_attention[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d      │ (None, 256)               │               0 │ concatenate[0][0]          │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d          │ (None, 256)               │               0 │ concatenate[0][0]          │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 512)               │               0 │ global_average_pooling1d[… │
│                               │                           │                 │ global_max_pooling1d[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 256)               │         131,328 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 256)               │           1,024 │ dense[0][0]                │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 256)               │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 717,904 (2.74 MB)

...

155/155 ━━━━━━━━━━━━━━━━━━━━ 87s 557ms/step - accuracy: 0.8486 - loss: 0.3562 - macro_f1_score: 0.8486 - val_accuracy: 0.7177 - val_loss: 0.6918 - val_macro_f1_score: 0.7177
Epoch 13/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step - accuracy: 0.8633 - loss: 0.3344 - macro_f1_score: 0.8633 
Epoch 13: macro_f1_score improved from 0.84184 to 0.85838, saving model to models_hi_task1_m2\best_model_hi_task1_m2.h5


155/155 ━━━━━━━━━━━━━━━━━━━━ 61s 394ms/step - accuracy: 0.8633 - loss: 0.3345 - macro_f1_score: 0.8633 - val_accuracy: 0.6831 - val_loss: 0.7920 - val_macro_f1_score: 0.6831
Epoch 14/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step - accuracy: 0.8756 - loss: 0.2948 - macro_f1_score: 0.8756 
Epoch 14: macro_f1_score improved from 0.85838 to 0.86706, saving model to models_hi_task1_m2\best_model_hi_task1_m2.h5


155/155 ━━━━━━━━━━━━━━━━━━━━ 64s 415ms/step - accuracy: 0.8755 - loss: 0.2949 - macro_f1_score: 0.8755 - val_accuracy: 0.7129 - val_loss: 0.7363 - val_macro_f1_score: 0.7129
Epoch 15/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step - accuracy: 0.8860 - loss: 0.2809 - macro_f1_score: 0.8860 
Epoch 15: macro_f1_score improved from 0.86706 to 0.88420, saving model to models_hi_task1_m2\best_model_hi_task1_m2.h5


155/155 ━━━━━━━━━━━━━━━━━━━━ 63s 409ms/step - accuracy: 0.8860 - loss: 0.2810 - macro_f1_score: 0.8860 - val_accuracy: 0.6823 - val_loss: 0.8258 - val_macro_f1_score: 0.6823
Restoring model weights from the end of the best epoch: 15.


39/39 ━━━━━━━━━━━━━━━━━━━━ 10s 161ms/step

Validation Results:
Precision: 0.6780
Recall: 0.6823
weighted F1 Score: 0.6800
macro F1 Score: 0.6072

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.77      0.79      0.78       883
        hate       0.45      0.43      0.44       357

    accuracy                           0.68      1240
   macro avg       0.61      0.61      0.61      1240
weighted avg       0.68      0.68      0.68      1240



In [12]:
#####################################
#TESTING

In [13]:
d2 = pd.read_csv('test_hi_l1.csv', engine='python', on_bad_lines='skip')
d2
d2 = d2.rename(columns={'key' : 'unique_id', 'sentence' : 'text'})
d2.to_csv('updated_test_hi_l1.csv', index=False)
# d2

# Convert annotator columns to numeric without replacing NaNs
d2[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']] = d2[
    ['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']
].apply(pd.to_numeric, errors='coerce')

# Compute 'label' from Hindi annotators using majority voting (ignore NaNs)
d2['label'] = (d2[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']].mean(axis=1, skipna=True) >= 0.5).astype(int)
d2


,text,unique_id,hi_a1,hi_a2,hi_a3,hi_a4,hi_a5,label
0,#BandraStation #SharadPawar #Muradabad अगर अभ...,question_1,NaN,0.0,0.0,0.0,NaN,0
1,#ConspiracyAgainstIndia सुन लो रे देश के गद्द...,question_1,NaN,0.0,0.0,0.0,NaN,0
2,#MarathaReservation : महाराष्ट्र में जश्न का म...,question_1,0.0,0.0,0.0,NaN,NaN,0
3,#RheaChakraborty aap Mahesh bhatt se madad kiy...,question_1,NaN,1.0,1.0,0.0,NaN,1
4,"#SecularMaskOfd हमे पता नहि क्या कहते है ,इतना...",question_1,0.0,NaN,1.0,0.0,NaN,0
...,...,...,...,...,...,...,...,...
1512,है हैवान लोग होते हैं इनको ओसामा लादेन खतरनाक ...,question_1,NaN,0.0,NaN,0.0,1.0,0
1513,"🇮🇳जय हिन्द,🇮🇳जय भारत,🇮🇳वन्दे मातरम🇮🇳🇮🇳 बघेल तु...",question_1,NaN,0.0,NaN,1.0,1.0,1
1514,🙏जय भीम 🙏 जो लोग हमें धमकियां देते है और संव...,question_1,0.0,NaN,NaN,0.0,0.0,0
1515,🚩🕉🚩 *इंसान के पास 😇दिमाग होना चाहिये* *💀खोपड़...,question_1,0.0,NaN,0.0,0.0,NaN,0


In [14]:
d2.loc[:, 'binary_label'] = d2['label'].apply(determine_binary_label)

# # Reorder columns
d2 = d2[['unique_id', 'text', 'binary_label', 'label']]

d2.to_csv('updated_test_hi_l1.csv', index=False)

d2.loc[:, 'text'] = d2['text'].apply(lambda x: normalize_text(x))
processed_list = []
for j in d2['text']:
    process = j.replace('...','')
    processed_list.append(process)

df_processed = pd.DataFrame(processed_list)
df_processed.columns = ['text']
df_processed.head(n=5)

X = list(df_processed['text'])
y = d2[['label']].values

X = tokenizer.texts_to_sequences(X)

# Padding
X = pad_sequences(X, padding='post', maxlen=max_len)

y = label_encoder.fit_transform(y.ravel())

y = to_categorical(y, num_classes=2)


In [15]:
test_results = evaluate_validation(trained_model, X, y)

print(r"\Test Results:")
print(f"Precision: {test_results['precision']:.4f}")
print(f"Recall: {test_results['recall']:.4f}")
print(f"weighted F1 Score: {test_results['f1_score_weighted']:.4f}")
print(f"macro F1 Score: {test_results['f1_score_macro']:.4f}")
print("\nClassification Report:")
print(test_results['classification_report'])

48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step
\Test Results:
Precision: 0.7363
Recall: 0.7152
weighted F1 Score: 0.7241
macro F1 Score: 0.6309

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.84      0.78      0.81      1159
        hate       0.41      0.50      0.45       358

    accuracy                           0.72      1517
   macro avg       0.63      0.64      0.63      1517
weighted avg       0.74      0.72      0.72      1517

